# MLB.com stats crawler
https://github.com/whyhugo/Data-Analysis-in-Sports
> Please input the URL which can get from https://www.mlb.com/stats

In [1]:
import requests as rq
import pandas as pd
from bs4 import BeautifulSoup as soup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from lxml import etree
import time

In [2]:
chrome_options = Options()
chrome_options.add_argument('--headless')

url = input('URL:')
driver = webdriver.Chrome('/programing/swiftx/chromedriver-win64/chromedriver.exe')
html = driver.get(url)
sp = soup(driver.page_source, 'html.parser')

C:\Users\user\AppData\Local\Temp\ipykernel_19084\1222985024.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/programing/swiftx/chromedriver-win64/chromedriver.exe')


get player index
```python
index_list = []
index = sp.find_all(class_ = 'index-3cdMSKi7')
for i in range(0, len(index), 2):
    index_list.append(index[i].text)
```

In [3]:
# get player name
def get_player_name():
    player_list = []
    player = sp.find_all(class_ = 'full-3fV3c9pF')
    for i in range(0, len(player), 2):
        player_list.append(player[i].text + ' ' + player[i+1].text)
    return player_list

In [4]:
# get player's position
def get_player_position():
    position_list = []
    position = sp.find_all(class_ = 'position-28TbwVOg')
    for i in position:
        position_list.append(i.text)
    return position_list

In [5]:
def get_column_name():
    click_standard_button()
    sp = soup(driver.page_source, 'html.parser')
    
    data_column_name = ['PLAYER', 'POSITION', 'TEAM']
    temp = sp.find_all(class_ = 'bui-text cellheader bui-text')
    for i in range(4, len(temp), 2):
        data_column_name.append(temp[i].text)
    temp = sp.find_all(class_ = 'bui-text cellheader selected-1vxxHvFg bui-text')
    for i in range(0, len(temp), 2):
        data_column_name.append(temp[i].text)

    click_expanded_button()
    sp = soup(driver.page_source, 'html.parser')
    temp = sp.find_all(class_ = 'bui-text cellheader bui-text')
    for i in range(4, len(temp), 2):
        data_column_name.append(temp[i].text)
        
    return data_column_name

In [6]:
# click standard button
def click_standard_button():
    time.sleep(3)
    driver.find_element('xpath', '//*[@id="stats-app-root"]/section/section/div[1]/div[2]/div/div[1]/div/div[1]/button').click()
    driver.find_element('xpath', '//*[@id="stats-app-root"]/section/section/div[1]/div[2]/div/div[1]/div/div[1]/button').click()
    time.sleep(3)
    sp = soup(driver.page_source, 'html.parser')

In [7]:
# click expanded button
def click_expanded_button():
    time.sleep(3)
    driver.find_element('xpath', '//*[@id="stats-app-root"]/section/section/div[1]/div[2]/div/div[1]/div/div[2]/button').click()
    driver.find_element('xpath', '//*[@id="stats-app-root"]/section/section/div[1]/div[2]/div/div[1]/div/div[2]/button').click()
    time.sleep(3)

In [8]:
# click next page button
def click_next_page_button():
    time.sleep(3)
    driver.find_element('xpath', '//*[@id="stats-app-root"]/section/section/div[3]/div[2]/div/div/div[2]/button').click()
    time.sleep(3)
    sp = soup(driver.page_source, 'html.parser')

In [9]:
# get standard data
def get_data():
    data = []
    
    click_standard_button()
    sp = soup(driver.page_source, 'html.parser')
    standard_data = []
    tt = sp.select('#stats-app-root tr td')
    for i in tt:
        standard_data.append(i.text)
    
    click_expanded_button()
    sp = soup(driver.page_source, 'html.parser')
    expanded_data = []
    tt = sp.select('#stats-app-root tr td')
    for i in tt:
        expanded_data.append(i.text)

    for i in range(len(standard_data)//17):
        data.extend(standard_data[i*17:i*17+17])
        data.extend(expanded_data[i*16+1:i*16+16])

    return data

```python
html = driver.get(url) 
# 有時候會 get 不完整，可以用這邊 reload 一次 XD
```

In [10]:
try:
    df_list = []
    while True:
        sp = soup(driver.page_source, 'html.parser')
        print('page start')
        time.sleep(2)
        name = get_player_name()
        time.sleep(2)
        position = get_player_position()
        time.sleep(2)
        current_data = get_data()
        time.sleep(2)
        for i in range(len(name)):
            row = []
            row.append(name[i])
            row.append(position[i])
            row.extend(current_data[i*32:i*32+32])
            df_list.append(row)
        click_next_page_button()
except:
    print('This is the last page :)')

page start
page start
This is the last page :)


In [11]:
df = pd.DataFrame(df_list, columns=get_column_name())
df

,PLAYER,POSITION,TEAM,G,AB,R,H,2B,3B,HR,...,GO/AO,XBH,TB,IBB,BABIP,ISO,AB/HR,BB/K,BB%,SO%
0,Robert Carson,P,NYM,14,0,1,0,0,0,0,...,-.--,0,0,0,.---,--,-.--,.---,1.000,.000
1,David Wright,3B,NYM,112,430,63,132,23,6,18,...,0.73,47,221,5,.340,.207,23.89,.696,.112,.161
2,Zach Lutz,X,NYM,15,20,2,6,2,0,0,...,1.67,2,8,0,.429,.100,-.--,1.000,.231,.231
3,Marlon Byrd,RF,NYM,117,425,61,121,26,5,21,...,0.75,52,220,2,.350,.233,20.24,.202,.054,.267
4,Josh Satin,1B,NYM,75,190,23,53,15,0,3,...,1.16,18,77,2,.379,.126,63.33,.536,.136,.253
5,Lucas Duda,LF,NYM,100,318,42,71,16,0,15,...,0.60,31,132,4,.276,.192,21.20,.539,.143,.266
6,Daniel Murphy,2B,NYM,161,658,92,188,38,4,13,...,0.89,55,273,2,.315,.129,50.62,.337,.046,.136
7,Justin Turner,X,NYM,86,200,12,56,13,1,2,...,1.29,16,77,1,.327,.105,100.00,.324,.051,.159
8,Andrew Brown,RF,NYM,68,150,16,34,5,0,7,...,1.06,12,60,0,.273,.173,21.43,.295,.079,.267
9,Anthony Recker,C,NYM,50,135,17,29,7,0,6,...,0.76,13,54,1,.280,.185,22.50,.265,.086,.325


In [ ]:
df.to_csv('mlb_stats.csv', index=False, encoding='utf-8')

In [ ]:
driver.quit()